In [0]:
import torch
from torchvision import datasets
import torchvision.transforms as transforms
import numpy as np

In [4]:

num_workers = 0

batch_size = 20

# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# choose the training and test datasets
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

Processing...
Done!


In [0]:
import torch.nn as nn
import torch.nn.functional as F

# define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # number of hidden nodes in each layer (512)
        hidden_1 = 512
        hidden_2 = 512
        # linear layer (784 -> hidden_1)
        self.fc1 = nn.Linear(28 * 28, hidden_1)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        # linear layer (n_hidden -> 10)
        self.fc3 = nn.Linear(hidden_2, 10)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # flatten image input
        x = x.view(-1, 28 * 28)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc2(x))
        # add dropout layer
        x = self.dropout(x)
        # add output layer
        x = self.fc3(x)
        return x

# initialize the NN
model = Net()

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [7]:
# number of epochs to train the model
n_epochs = 50

model.train() # prep model for training

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
             
    # print training statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss
        ))

Epoch: 1 	Training Loss: 0.815939
Epoch: 2 	Training Loss: 0.320701
Epoch: 3 	Training Loss: 0.248348
Epoch: 4 	Training Loss: 0.201966
Epoch: 5 	Training Loss: 0.169304
Epoch: 6 	Training Loss: 0.146618
Epoch: 7 	Training Loss: 0.129873
Epoch: 8 	Training Loss: 0.115656
Epoch: 9 	Training Loss: 0.103717
Epoch: 10 	Training Loss: 0.094061
Epoch: 11 	Training Loss: 0.087395
Epoch: 12 	Training Loss: 0.080343
Epoch: 13 	Training Loss: 0.073384
Epoch: 14 	Training Loss: 0.069348
Epoch: 15 	Training Loss: 0.064759
Epoch: 16 	Training Loss: 0.058662
Epoch: 17 	Training Loss: 0.055572
Epoch: 18 	Training Loss: 0.052114
Epoch: 19 	Training Loss: 0.049507
Epoch: 20 	Training Loss: 0.045669
Epoch: 21 	Training Loss: 0.043737
Epoch: 22 	Training Loss: 0.041296
Epoch: 23 	Training Loss: 0.038904
Epoch: 24 	Training Loss: 0.036927
Epoch: 25 	Training Loss: 0.034867
Epoch: 26 	Training Loss: 0.032423
Epoch: 27 	Training Loss: 0.030553
Epoch: 28 	Training Loss: 0.029724
Epoch: 29 	Training Loss: 0.0

In [8]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval() # prep model for evaluation

for data, target in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate and print avg test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.054438

Test Accuracy of     0: 99% (971/980)
Test Accuracy of     1: 99% (1128/1135)
Test Accuracy of     2: 98% (1015/1032)
Test Accuracy of     3: 98% (990/1010)
Test Accuracy of     4: 98% (966/982)
Test Accuracy of     5: 98% (877/892)
Test Accuracy of     6: 98% (943/958)
Test Accuracy of     7: 98% (1010/1028)
Test Accuracy of     8: 97% (947/974)
Test Accuracy of     9: 97% (988/1009)

Test Accuracy (Overall): 98% (9835/10000)
